In [1]:
import utils
import requests

text_by_file = utils.load_files('./ustawy/*')

1. Use SpaCy [tokenizer API](https://spacy.io/api/tokenizer) to tokenize the text from the law corpus.

In [33]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish

for file,text in text_by_file.items():
    text_by_file[file] = ' '.join(text.split())
    

nlp = Polish()
tokenizer = nlp.tokenizer
files = tokenizer.pipe(text_by_file.values())

2. Compute **bigram** counts of downcased tokens.  Given the sentence: "The quick brown fox jumps over the
   lazy dog.", the bigram counts are as follows:
   1. "the quick": 1
   1. "quick brown": 1
   1. "brown fox": 1
   1. ...
   1. "dog .": 1

In [30]:
from collections import Counter

bigrams = []

for doc in files:
    words = [token.text.lower() for token in doc]
    bigrams += list(zip(words[:-1],words[1:]))

freq = Counter(bigrams)
freq

Counter({('dz', '.'): 8885,
         ('.', 'u'): 8174,
         ('u', '.'): 8134,
         ('.', 'z'): 5404,
         ('z', '2000'): 1506,
         ('2000', 'r'): 2271,
         ('r', '.'): 33015,
         ('.', 'nr'): 21433,
         ('nr', '31'): 175,
         ('31', ','): 285,
         (',', 'poz'): 43188,
         ('poz', '.'): 45198,
         ('.', '381'): 17,
         ('381', 'ustawa'): 1,
         ('ustawa', 'z'): 1190,
         ('z', 'dnia'): 9527,
         ('dnia', '2'): 149,
         ('2', 'marca'): 22,
         ('marca', '2000'): 43,
         ('.', 'o'): 7693,
         ('o', 'zmianie'): 1299,
         ('zmianie', 'ustawy'): 850,
         ('ustawy', 'o'): 1444,
         ('o', 'przeciwdziałaniu'): 109,
         ('przeciwdziałaniu', 'praktykom'): 29,
         ('praktykom', 'monopolistycznym'): 38,
         ('monopolistycznym', 'i'): 12,
         ('i', 'ochronie'): 22,
         ('ochronie', 'interesów'): 12,
         ('interesów', 'konsumentów'): 53,
         ('konsumentów', 'ar

3. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries **after**
   computing the bigram counts.

In [31]:
freq = dict(filter(lambda elem: elem[0][0].isalpha() and elem[0][1].isalpha(), freq.items()))
freq

{('ustawa', 'z'): 1190,
 ('z', 'dnia'): 9527,
 ('o', 'zmianie'): 1299,
 ('zmianie', 'ustawy'): 850,
 ('ustawy', 'o'): 1444,
 ('o', 'przeciwdziałaniu'): 109,
 ('przeciwdziałaniu', 'praktykom'): 29,
 ('praktykom', 'monopolistycznym'): 38,
 ('monopolistycznym', 'i'): 12,
 ('i', 'ochronie'): 22,
 ('ochronie', 'interesów'): 12,
 ('interesów', 'konsumentów'): 53,
 ('konsumentów', 'art'): 3,
 ('w', 'ustawie'): 5180,
 ('ustawie', 'z'): 3661,
 ('w', 'art'): 32042,
 ('wprowadza', 'się'): 1849,
 ('się', 'następujące'): 1855,
 ('następujące', 'zmiany'): 1808,
 ('otrzymuje', 'brzmienie'): 9553,
 ('prezes', 'jest'): 4,
 ('jest', 'centralnym'): 26,
 ('centralnym', 'organem'): 51,
 ('organem', 'administracji'): 85,
 ('administracji', 'rządowej'): 988,
 ('po', 'ust'): 1541,
 ('dodaje', 'się'): 8195,
 ('się', 'ust'): 3484,
 ('w', 'brzmieniu'): 7280,
 ('prezes', 'rady'): 602,
 ('rady', 'ministrów'): 1168,
 ('ministrów', 'sprawuje'): 7,
 ('sprawuje', 'nadzór'): 132,
 ('nadzór', 'nad'): 517,
 ('nad', 'dzia

4. Use [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) to compute the measure 
   for all pairs of words. 

In [34]:
words = []

for doc in files:
    words += [token.text.lower() for token in doc if token.text.isalpha()]

word_freq = Counter(words)
word_freq

Counter({'dz': 8885,
         'u': 9153,
         'z': 82443,
         'r': 33177,
         'nr': 44950,
         'poz': 45224,
         'ustawa': 3235,
         'dnia': 17954,
         'marca': 783,
         'o': 64776,
         'zmianie': 1518,
         'ustawy': 13099,
         'przeciwdziałaniu': 216,
         'praktykom': 43,
         'monopolistycznym': 39,
         'i': 90009,
         'ochronie': 1054,
         'interesów': 236,
         'konsumentów': 328,
         'art': 83804,
         'w': 201223,
         'ustawie': 5478,
         'lutego': 608,
         'wprowadza': 1999,
         'się': 45886,
         'następujące': 2276,
         'zmiany': 4016,
         'a': 17133,
         'ust': 53636,
         'otrzymuje': 9835,
         'brzmienie': 10576,
         'prezes': 2497,
         'jest': 13197,
         'centralnym': 129,
         'organem': 683,
         'administracji': 2941,
         'rządowej': 1097,
         'b': 6849,
         'po': 13547,
         'dodaje': 8423,


In [35]:
word_total = sum(word_freq.values())
bigram_total = sum(freq.values())

In [36]:
from math import log2

def pmi(a: str, b: str) -> float:
    pa = word_freq[a]/word_total
    pb = word_freq[b]/word_total
    pab = freq[(a,b)]/bigram_total
    return log2(pab/(pa*pb))

In [37]:
bigram_pmi = {bigram: pmi(*bigram) for bigram in freq.keys()}
bigram_pmi

{('ustawa', 'z'): 4.375256412169623,
 ('z', 'dnia'): 4.9038491447023995,
 ('o', 'zmianie'): 5.9412278119895365,
 ('zmianie', 'ustawy'): 7.635363832038612,
 ('ustawy', 'o'): 2.9846841411281777,
 ('o', 'przeciwdziałaniu'): 5.179294994570579,
 ('przeciwdziałaniu', 'praktykom'): 13.825998672407135,
 ('praktykom', 'monopolistycznym'): 16.68543047402437,
 ('monopolistycznym', 'i'): 3.9909485727211806,
 ('i', 'ochronie'): 0.10916075786235552,
 ('ochronie', 'interesów'): 7.809830233867229,
 ('interesów', 'konsumentów'): 11.636895334728404,
 ('konsumentów', 'art'): -0.9781510553075033,
 ('w', 'ustawie'): 4.450037075451804,
 ('ustawie', 'z'): 5.236649058453354,
 ('w', 'art'): 3.1436793383832353,
 ('wprowadza', 'się'): 6.550870258441064,
 ('się', 'następujące'): 6.368322134983079,
 ('następujące', 'zmiany'): 9.845520500302962,
 ('otrzymuje', 'brzmienie'): 8.738692284743909,
 ('prezes', 'jest'): -0.8247325584143633,
 ('jest', 'centralnym'): 6.150460010231687,
 ('centralnym', 'organem'): 11.3946262

5. Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [38]:
top_10 = list(sorted(bigram_pmi.items(), key=lambda item: item[1], reverse=True))[:10]
top_10

[(('młody', 'naukowiec'), 22.149169934145128),
 (('młodym', 'naukowcom'), 22.149169934145128),
 (('osiągniętymi', 'efektami'), 22.149169934145128),
 (('del', 'credere'), 22.149169934145128),
 (('doktorów', 'habilitowanych'), 22.149169934145128),
 (('fenolami', 'lotnymi'), 22.149169934145128),
 (('ostrzeżony', 'strzałami'), 22.149169934145128),
 (('strzałami', 'ostrzegawczymi'), 22.149169934145128),
 (('płynami', 'infuzyjnymi'), 22.149169934145128),
 (('dzierżawionego', 'obowdu'), 22.149169934145128)]

6. Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5
   occurrences).

In [39]:
bigram_5_pmi = {bigram: pmi(*bigram) for (bigram,count) in freq.items() if count > 4}
top_10_5 = list(sorted(bigram_5_pmi.items(), key=lambda item: item[1], reverse=True))[:10]
top_10_5

[(('obiegów', 'chłodzących'), 19.827241839257766),
 (('otworami', 'wiertniczymi'), 19.827241839257766),
 (('ręcznego', 'miotacza'), 19.827241839257766),
 (('świeckie', 'przygotowujące'), 19.827241839257766),
 (('stajnią', 'wyścigową'), 19.827241839257766),
 (('młynki', 'młotkowe'), 19.827241839257766),
 (('młyny', 'kulowe'), 19.827241839257766),
 (('najnowszych', 'zdobyczy'), 19.827241839257766),
 (('klęskami', 'żywiołowymi'), 19.827241839257766),
 (('zaszkodzić', 'wynikom'), 19.827241839257766)]

7. Use [KRNNT](https://hub.docker.com/r/djstrong/krnnt2) or Clarin-PL API(https://ws.clarin-pl.eu/tager.shtml) to tag and lemmatize the corpus.

8. Using the tagged corpus compute bigram statistic for the tokens containing:
   a. lemmatized, downcased word
   b. morphosyntactic category of the word (subst, fin, adj, etc.)
9. For example: "Ala ma kota", which is tagged as:
   ```
   Ala	none
           Ala	subst:sg:nom:f	disamb
   ma	space
           mieć	fin:sg:ter:imperf	disamb
   kota	space
           kot	subst:sg:acc:m2	disamb
   .	none
           .	interp	disamb
   ```
   the algorithm should return the following bigrams: `ala:subst mieć:fin` and `mieć:fin kot:subst`.

10. Compute the same statistics as for the non-lemmatized words (i.e. PMI) and print top-10 entries with at least 5 occurrences.

11. Compute **trigram** counts for both corpora and perform the same filtering.

12. Use PMI (with 5 occurrence threshold) to compute top 10 results for the trigrams. Devise a method for computing the values, based on the
   results for bigrams.

13. Create a table comparing the results for copora without and with tagging and lemmatization (separate table for bigrams and trigrams).

14. Answer the following questions:
      1. Why do we have to filter the bigrams, rather than the token sequence?
      1. Which method works better for the bigrams and which for the trigrams?
      1. What types of expressions are discovered by the methods.
      1. Can you devise a different type of filtering that would yield better results?